In [1]:
#This code is to estimate ATE based on doubly robust methods 
#  DR method 1: AIPW
#  DR method 2: TMLE 


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.special import expit as sigmoid
from scipy.special import logit


np.random.seed(123321)
# Corr_matrix
def generate_corr_matrix(k, seed = None):
    if seed is not None:
        np.random.seed(seed)
    
    A = np.random.randn(k, k)
    A2 = A @ A.T   # Positive matrix 
    D = np.sqrt(np.diag(A2))
    corr_mat = A2 / np.outer(D, D)
    np.fill_diagonal(corr_mat, 1.0)
    
    return corr_mat

# Generate data.frame with [Y, D, X]
def generate_data(n, k):
    theta = 0.5      # True ATE
    b = 1 / np.arange(1, k + 1)  # Coefficients for X
    mu = np.zeros(k)  # Mean vector for multivariate normal
    cov = generate_corr_matrix(k)  # Correlation matrix
    # Predictors: n by k
    X = np.random.multivariate_normal(mu, cov, n)

    score = X @ b
    p_D = 1 / (1 + np.exp(-score))  # Propensity score (logistic)
    D = np.random.binomial(1, p_D)  # Binary D

    # Define structural outcome model
    g = np.cos(score)**2
    Y = theta  * D + g + np.random.normal(0, 1, n)

    # DataFrame [Y, D, X]
    columns = [f'X{i+1}' for i in range(k)]
    df = pd.DataFrame(X, columns=columns)
    df['D'] = D
    df['Y'] = Y
    
    return df

# Geneate data with n=1000, k=8
df = generate_data(1000, 8)
df2 = df.copy()
print(df2.head(10))
theta = 0.5      # True ATE
print('\n True ATE:', theta)

         X1        X2        X3        X4        X5        X6        X7  \
0  0.171745 -1.061727  0.266644 -0.093521 -1.695077  1.223727  0.086541   
1 -0.582003 -0.460072  0.641401  0.256456 -0.649026  0.038970  0.389041   
2 -0.390587 -0.948942 -0.457175  0.601283 -0.012413 -0.248519  0.751453   
3  0.258854 -0.197598  0.559316 -1.185145  0.582350 -0.028826 -1.128971   
4 -0.601644  0.739605  0.835163  0.435640 -1.943612 -1.120272 -1.019158   
5  0.626916 -0.610890 -0.392059  0.310769  0.148834  0.821127  0.987841   
6  1.225196  0.223089  1.888222  1.934322 -1.271216  1.291793  0.358159   
7  0.257843  0.609101 -0.688733  0.256400  0.631261 -0.179882  0.896016   
8  2.164947 -0.800811 -2.411401 -2.608424  1.602401 -2.471738 -1.638505   
9 -0.243146 -1.562291 -0.268241 -1.207587  0.411936  0.462073 -0.464335   

         X8  D         Y  
0  0.912584  0  0.905900  
1  0.131072  0  1.374393  
2 -0.751875  0  1.618586  
3  0.126175  0  1.236549  
4  1.673362  0  0.303016  
5 -1.072360 

In [3]:
#OLS estiamte 
XD_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'D']  # Covariates + D
X_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']  # Covariates only
Y_cols = ['Y']

X = sm.add_constant(df[XD_cols], prepend=True)
mod_ols = sm.OLS(df['Y'], X)
res_ols = mod_ols.fit()
print(res_ols.summary())

#Statistics of OLS estimate 
theta_ols = res_ols.params['D']
bias_ols = np.abs(theta_ols - theta)
pct_bias_ols = ((theta_ols - theta) / theta) * 100
print('\nOLS Estimates:', theta_ols)
print('Bias of OLS Estimates:', bias_ols)
print('Percentage Bias of OLS Estimates:', pct_bias_ols, '%')

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.078
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     9.290
Date:                Tue, 15 Apr 2025   Prob (F-statistic):           1.26e-13
Time:                        02:43:36   Log-Likelihood:                -1494.4
No. Observations:                1000   AIC:                             3009.
Df Residuals:                     990   BIC:                             3058.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5044      0.050     10.105      0.0

In [4]:
#TMLE method 
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.special import expit as sigmoid
from scipy.special import logit

if df['Y'].between(0, 1).all():
    outcome = 'binary'
    df['Y_prob'] = df['Y']    # Binary Y
else:
    outcome = "continuous"
    Y_min = df['Y'].min()
    Y_max = df['Y'].max()
    df['Y_prob'] = (df['Y'] - Y_min) / (Y_max - Y_min)
    #df['Y_prob'] = df['Y_prob'].clip(0.001, 0.999)  #Avoid 0 or 1

# Step 2: Define columns 
XD_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'D']  # Covariates + D
X_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']  # Covariates only
Y_cols = ['Y_prob']


In [5]:
# Step 3: Outcome model 
X = sm.add_constant(df[XD_cols], prepend=True)
mod_Y = sm.GLM(df['Y_prob'], X, 
              family=sm.families.Binomial(link=sm.families.links.Logit())).fit()

#Predict Y_hat Y1_hat, Y0_hat
df['Y_hat'] = mod_Y.predict(X.values)

df['D0'] = 0 
df['D1'] = 1  
X1 = sm.add_constant(df[X_cols + ['D1']], has_constant='add', prepend=True)
df['Y1_hat'] = mod_Y.predict(X1.values)   # X1 does not work as D1 is not D

X0 = sm.add_constant(df[X_cols + ['D0']], has_constant='add', prepend=True)
df['Y0_hat'] = mod_Y.predict(X0.values)

print(df.head(10))

         X1        X2        X3        X4        X5        X6        X7  \
0  0.171745 -1.061727  0.266644 -0.093521 -1.695077  1.223727  0.086541   
1 -0.582003 -0.460072  0.641401  0.256456 -0.649026  0.038970  0.389041   
2 -0.390587 -0.948942 -0.457175  0.601283 -0.012413 -0.248519  0.751453   
3  0.258854 -0.197598  0.559316 -1.185145  0.582350 -0.028826 -1.128971   
4 -0.601644  0.739605  0.835163  0.435640 -1.943612 -1.120272 -1.019158   
5  0.626916 -0.610890 -0.392059  0.310769  0.148834  0.821127  0.987841   
6  1.225196  0.223089  1.888222  1.934322 -1.271216  1.291793  0.358159   
7  0.257843  0.609101 -0.688733  0.256400  0.631261 -0.179882  0.896016   
8  2.164947 -0.800811 -2.411401 -2.608424  1.602401 -2.471738 -1.638505   
9 -0.243146 -1.562291 -0.268241 -1.207587  0.411936  0.462073 -0.464335   

         X8  D         Y    Y_prob     Y_hat  D0  D1    Y1_hat    Y0_hat  
0  0.912584  0  0.905900  0.529910  0.438268   0   1  0.520893  0.438268  
1  0.131072  0  1.374393

In [6]:
# Step 4, PS model 
X = sm.add_constant(df[X_cols], prepend=True)
mod_PS = sm.GLM(df['D'], X, family=sm.families.Binomial()).fit()

ps_hat = mod_PS.predict(X)
df['ps_hat'] = ps_hat

# Define clever covariates
df['Haw'] = df['D']/df['ps_hat'] - (1-df['D'])/(1-df['ps_hat'])
df['H1w'] = (1/df['ps_hat'])
df['H0w'] = -1/(1-df['ps_hat'])

print(df.head(10))

         X1        X2        X3        X4        X5        X6        X7  \
0  0.171745 -1.061727  0.266644 -0.093521 -1.695077  1.223727  0.086541   
1 -0.582003 -0.460072  0.641401  0.256456 -0.649026  0.038970  0.389041   
2 -0.390587 -0.948942 -0.457175  0.601283 -0.012413 -0.248519  0.751453   
3  0.258854 -0.197598  0.559316 -1.185145  0.582350 -0.028826 -1.128971   
4 -0.601644  0.739605  0.835163  0.435640 -1.943612 -1.120272 -1.019158   
5  0.626916 -0.610890 -0.392059  0.310769  0.148834  0.821127  0.987841   
6  1.225196  0.223089  1.888222  1.934322 -1.271216  1.291793  0.358159   
7  0.257843  0.609101 -0.688733  0.256400  0.631261 -0.179882  0.896016   
8  2.164947 -0.800811 -2.411401 -2.608424  1.602401 -2.471738 -1.638505   
9 -0.243146 -1.562291 -0.268241 -1.207587  0.411936  0.462073 -0.464335   

         X8  D         Y    Y_prob     Y_hat  D0  D1    Y1_hat    Y0_hat  \
0  0.912584  0  0.905900  0.529910  0.438268   0   1  0.520893  0.438268   
1  0.131072  0  1.3743

In [7]:
# Step 5, Derive Epsilon parameters  
mod_eps = sm.GLM(df['Y_prob'], df[['Haw']], 
                 family=sm.families.Binomial(),  
                 offset=logit(df['Y_hat'])).fit() 
epsilon = mod_eps.params['Haw']
print(f'Values of epsilon is: {epsilon: 4f}')

# Using two parameters
X_eps = df[['H0w', 'H1w']] 
mod_eps = sm.GLM(df['Y_prob'], X_eps, 
                 family=sm.families.Binomial(),
                 offset=logit(df['Y_hat'])).fit()
# Extract epsilon parameters
eps0 = mod_eps.params['H0w']
eps1 = mod_eps.params['H1w']
print(f'Epsilon0 (H0w): {eps0:.4f}, Epsilon1 (H1w): {eps1:.4f}')



Values of epsilon is: -0.003870
Epsilon0 (H0w): -0.0008, Epsilon1 (H1w): -0.0052


In [8]:
# Step 6: update Y1_hat and Y0_hat
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def logit(p):
    return np.log(p / (1 - p))
    
df['Y1_update'] = sigmoid(logit(df['Y1_hat']) + eps1 * df['H1w'])
df['Y0_update'] = sigmoid(logit(df['Y0_hat']) + eps0 * df['H0w'])

# Scale continuous predictions
if outcome == "continuous":
    df['Y1_update_scaled'] = (Y_max - Y_min) * df['Y1_update'] + Y_min
    df['Y0_update_scaled'] = (Y_max - Y_min) * df['Y0_update'] + Y_min
elif outcome == "binary":
    df['Y1_update_scaled'] = df['Y1_update']
    df['Y0_update_scaled'] = df['Y0_update']
else:
    raise ValueError("Outcome must be either 'binary' or 'continuous'.")

print(df.head(10))

         X1        X2        X3        X4        X5        X6        X7  \
0  0.171745 -1.061727  0.266644 -0.093521 -1.695077  1.223727  0.086541   
1 -0.582003 -0.460072  0.641401  0.256456 -0.649026  0.038970  0.389041   
2 -0.390587 -0.948942 -0.457175  0.601283 -0.012413 -0.248519  0.751453   
3  0.258854 -0.197598  0.559316 -1.185145  0.582350 -0.028826 -1.128971   
4 -0.601644  0.739605  0.835163  0.435640 -1.943612 -1.120272 -1.019158   
5  0.626916 -0.610890 -0.392059  0.310769  0.148834  0.821127  0.987841   
6  1.225196  0.223089  1.888222  1.934322 -1.271216  1.291793  0.358159   
7  0.257843  0.609101 -0.688733  0.256400  0.631261 -0.179882  0.896016   
8  2.164947 -0.800811 -2.411401 -2.608424  1.602401 -2.471738 -1.638505   
9 -0.243146 -1.562291 -0.268241 -1.207587  0.411936  0.462073 -0.464335   

         X8  D         Y  ...    Y1_hat    Y0_hat    ps_hat       Haw  \
0  0.912584  0  0.905900  ...  0.520893  0.438268  0.303659 -1.436079   
1  0.131072  0  1.374393  ..

In [9]:
# Step 7: Compute TMLE estimate
theta_est = df['Y1_update_scaled'] - df['Y0_update_scaled']
theta_tmle = theta_est.mean()
bias_tmle = np.abs(theta_tmle  -theta)
pct_bias_tmle = ((theta_tmle  - theta) / theta) * 100

print('\nTMLE Estimates:', theta_tmle )
print('Bias of TMLE Estimates:', bias_tmle)
print('Percentage Bias of TMLE Estimates:', pct_bias_tmle, '%')



TMLE Estimates: 0.5196510734715327
Bias of TMLE Estimates: 0.01965107347153272
Percentage Bias of TMLE Estimates: 3.930214694306544 %


In [10]:
#AIPW method 
import numpy as np
import pandas as pd
import statsmodels.api as sm

XD_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'D']  # Covariates + D
X_cols = ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8']  
    
# Step 1: Outcome model (G-computation)
X_outcome = sm.add_constant(df2[XD_cols], prepend=True)
mod_Y = sm.GLM(df2['Y'], X_outcome, family=sm.families.Gaussian()).fit()

df2['D1'] = 1
df2['D0'] = 0
X1 = sm.add_constant(df2[X_cols + ['D1']], has_constant='add', prepend=True)
df2['Y1_hat'] = mod_Y.predict(X1)

#X0 = sm.add_constant(df[X_cols + ['D0']], prepend=True)
X0 = sm.add_constant(df2[X_cols + ['D0']], has_constant='add', prepend=True)
df2['Y0_hat'] = mod_Y.predict(X0)

print(df2.head(8))

         X1        X2        X3        X4        X5        X6        X7  \
0  0.171745 -1.061727  0.266644 -0.093521 -1.695077  1.223727  0.086541   
1 -0.582003 -0.460072  0.641401  0.256456 -0.649026  0.038970  0.389041   
2 -0.390587 -0.948942 -0.457175  0.601283 -0.012413 -0.248519  0.751453   
3  0.258854 -0.197598  0.559316 -1.185145  0.582350 -0.028826 -1.128971   
4 -0.601644  0.739605  0.835163  0.435640 -1.943612 -1.120272 -1.019158   
5  0.626916 -0.610890 -0.392059  0.310769  0.148834  0.821127  0.987841   
6  1.225196  0.223089  1.888222  1.934322 -1.271216  1.291793  0.358159   
7  0.257843  0.609101 -0.688733  0.256400  0.631261 -0.179882  0.896016   

         X8  D         Y  D1  D0    Y1_hat    Y0_hat  
0  0.912584  0  0.905900   1   0  0.845841  0.294532  
1  0.131072  0  1.374393   1   0  1.035547  0.484237  
2 -0.751875  0  1.618586   1   0  1.072489  0.521180  
3  0.126175  0  1.236549   1   0  0.980753  0.429444  
4  1.673362  0  0.303016   1   0  0.860435  0.309

In [11]:
#Step 2: PS model 
X_ps = sm.add_constant(df2[X_cols], prepend=True)
mod_PS = sm.GLM(df2['D'], X_ps, family=sm.families.Binomial()).fit()
df2['ps_hat'] = mod_PS.predict(X_ps)

print(df2.head(8))

         X1        X2        X3        X4        X5        X6        X7  \
0  0.171745 -1.061727  0.266644 -0.093521 -1.695077  1.223727  0.086541   
1 -0.582003 -0.460072  0.641401  0.256456 -0.649026  0.038970  0.389041   
2 -0.390587 -0.948942 -0.457175  0.601283 -0.012413 -0.248519  0.751453   
3  0.258854 -0.197598  0.559316 -1.185145  0.582350 -0.028826 -1.128971   
4 -0.601644  0.739605  0.835163  0.435640 -1.943612 -1.120272 -1.019158   
5  0.626916 -0.610890 -0.392059  0.310769  0.148834  0.821127  0.987841   
6  1.225196  0.223089  1.888222  1.934322 -1.271216  1.291793  0.358159   
7  0.257843  0.609101 -0.688733  0.256400  0.631261 -0.179882  0.896016   

         X8  D         Y  D1  D0    Y1_hat    Y0_hat    ps_hat  
0  0.912584  0  0.905900   1   0  0.845841  0.294532  0.303659  
1  0.131072  0  1.374393   1   0  1.035547  0.484237  0.303956  
2 -0.751875  0  1.618586   1   0  1.072489  0.521180  0.267654  
3  0.126175  0  1.236549   1   0  0.980753  0.429444  0.399044  

In [12]:
# Step 3: AIPW estimate
#G-computation 
df2['G_comp'] = df2['Y1_hat'] - df2['Y0_hat']
# Adj
df2['adj'] = (df2['D'] * (df2['Y'] - df2['Y1_hat']) / df2['ps_hat']) - (
              (1 - df2['D']) * (df2['Y'] - df2['Y0_hat']) / (1 - df2['ps_hat']))


In [13]:
# AIPW statistics 
theta_est2 = df2['G_comp'] + df2['adj']
theta_aipw = theta_est2.mean()
bias_aipw = np.abs(theta_aipw  - theta)
pct_bias_aipw = ((theta_aipw - theta) / theta) * 100


print('\nAIPW Estimate:', theta_aipw )
print('Bias of AIPW Estimate:', bias_aipw)
print('Percentage Bias of AIPW Estimate:', pct_bias_aipw, '%')



AIPW Estimate: 0.5104665019240455
Bias of AIPW Estimate: 0.010466501924045524
Percentage Bias of AIPW Estimate: 2.0933003848091047 %


In [14]:
# Comparison 
results_comparison = pd.DataFrame({
    'Method': ['OLS', 'TMLE', 'AIPW'],
    'Estimate': [theta_ols, theta_tmle, theta_aipw],
    'Bias': [bias_ols, bias_tmle, bias_aipw],
    'Percentage Bias': [pct_bias_ols, pct_bias_tmle, pct_bias_aipw],
    'True Effect': [theta, theta, theta]  # For reference
})
results_comparison 

,Method,Estimate,Bias,Percentage Bias,True Effect
0,OLS,0.551309,0.051309,10.261837,0.5
1,TMLE,0.519651,0.019651,3.930215,0.5
2,AIPW,0.510467,0.010467,2.093300,0.5


### TMLE reference
Laan, Mark J van der, and Daniel Rubin. 2006. “Targeted Maximum Likelihood Learning.” The International Journal of Biostatistics 2 (1): 11.

Schuler, Megan S, and Sherri Rose. 2017. “Targeted Maximum Likelihood Estimation for Causal Inference in Observational Studies.” American Journal of Epidemiology 185 (1): 65–73. 

### AIPW reference 
Bang, Heejung, and James M Robins. 2005. “Doubly Robust Estimation in Missing Data and Causal Inference Models.” Biometrics 61 (4): 962–73. 

Glynn, Adam N, and Kevin M Quinn. 2010. “An Introduction to the Augmented Inverse Propensity Weighted Estimator.” Political Analysis 18 (1): 36–56. 
